In [1]:
from konlpy.tag import Twitter
from sklearn.decomposition import NMF
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import normalize
from scipy.sparse import dok_matrix
from collections import Counter
import numpy as np
import json
import pickle

twitter = Twitter()

In [2]:
with open("data/preprocessed_data.pkl", 'rb') as f:
    preprocessed_data = pickle.load(f)

In [3]:
# NMF
K = 10
nmf = NMF(n_components=K, max_iter=1000, alpha=0.1)
W = nmf.fit_transform(preprocessed_data['tdm'])
H = nmf.components_

# 각 토픽별 키워드 출력
for k in range(K):
    print(f"{k}th topic")
    for index in H[k].argsort()[::-1][:20]:
        print(preprocessed_data['index2voca'][index], end=" ")
    print("\n")

0th topic
대통령 박근혜 청와대 탄핵 파면 삼성동 사저 서울 에서 헌법재판소 결정 인용 지지자 오후 받다 떠나다 않다 입장 대한 검찰 

1th topic
대표 민주당 대선 에서 문재인 후보 더불다 지사 주자 경선 탈당 국민의당 안희정 정당 이재명 바르다 개헌 토론회 김종인 통합 

2th topic
에서 탄핵 집회 서울 열리다 오후 박근혜 태극기 광장 광화문 반대 참가자 경찰 국민 촛불집회 퇴진 기각 행동 운동 촉구 

3th topic
시간 대선 지원 출마 억원 보통사람 인상 소환 청춘 황교안 검찰 정부 금리 공개 변경 주택 문재인 한국 이상 출시 

4th topic
탄핵 선고 심판 헌재 헌법재판소 결정 대통령 인용 재판관 박근혜 오전 기각 대한 결과 국회 평의 앞두다 최종 사건 이후 

5th topic
국민 않다 없다 우리 에서 대한민국 아니다 결정 헌재 정치 탄핵 사람 생각 헌법 통합 지금 촛불 사회 민주주의 국가 

6th topic
대행 권한 대선 에서 출마 대통령 황교안 재판관 정미 선거 않다 후보 국정 결정 지정 관리 퇴임 선거일 까지 국무회의 

7th topic
수사 특검 검찰 대통령 결과 발표 받다 조사 혐의 에서 없다 최순실 박영수 않다 사실 대한 삼성 변호사 관련 사건 

8th topic
의원 한국 자유 탄핵 정당 바르다 대선 에서 국회 지사 보수 출마 김진태 친박 위원장 후보 정치 밝히다 없다 선언 

9th topic
에서 사드 중국 한국 정부 경제 배치 미국 상황 북한 않다 대한 정책 없다 관련 시장 따르다 문제 회의 정치 



In [8]:
from sklearn.manifold import TSNE

# select random index
selectNum = 2000
randIndex = np.random.choice(W.shape[0], selectNum, replace=False)
randIndex.sort()

tsne = TSNE(n_components=2, init='pca', verbose=1)
W2d = tsne.fit_transform(W[randIndex, :])
topicIndex = [v.argmax() for v in W[randIndex, :]]

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 2000 samples in 0.003s...
[t-SNE] Computed neighbors for 2000 samples in 0.173s...
[t-SNE] Computed conditional probabilities for sample 1000 / 2000
[t-SNE] Computed conditional probabilities for sample 2000 / 2000
[t-SNE] Mean sigma: 0.007972
[t-SNE] KL divergence after 250 iterations with early exaggeration: 70.951729
[t-SNE] Error after 1000 iterations: 0.985381


In [11]:
from bokeh.models import HoverTool
from bokeh.palettes import Category20
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, ColumnDataSource
output_notebook()

# 사용할 툴들
tools_to_show = 'hover,box_zoom,pan,save,reset,wheel_zoom'
p = figure(plot_width=720, plot_height=480, tools=tools_to_show)

source = ColumnDataSource(data={
    'x': W2d[:, 0],
    'y': W2d[:, 1],
    'id': [i for i in randIndex],
    'document': [preprocessed_data['raw_text'][randInd][:100] for randInd in randIndex],  # 해당 기사
    'topic': [str(i) for i in topicIndex],  # 토픽 번호
    'color': [Category20[K][i] for i in topicIndex]
})
p.circle(
    'x', 'y',
    source=source,
    legend='topic',
    color='color',
    fill_alpha=0.7,
    line_alpha=0.7)

# 몇가지 interaction
p.legend.location = "top_left"
hover = p.select(dict(type=HoverTool))
hover.tooltips = [("Topic", "@topic"), ('id', '@id'), ("Article", "@document")]
hover.mode = 'mouse'

show(p)

Loading BokehJS ...